<a href="https://colab.research.google.com/github/QuakFooLee/python/blob/main/Supply_Chain_Sustainability_ESG_Reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supply Chain Sustainability Reporting with Python

4 Steps to build an ESG reporting focusing on CO2 emissions of the Distribution Network

Reference:
- [Supply Chain Sustainability Reporting with Python](https://towardsdatascience.com/supply-chain-sustainability-reporting-with-python-161c1f63f267)
- [Github](https://github.com/samirsaci/supply-chain-sustainability/blob/main/Transportation%20CO2%20Emissions%20Reporting.ipynb)

**SUMMARY**

I. CO2 Emissions Formula for Transportation
Calculate the CO2 emissions of multi-modal transportation ?

II. Data Processing with Python

List of the datasets needed to build your report
1. Import Shipped Order Lines
All the purchased orders of your customers that have been shipped
2. Add Unit of Measure Conversions
Merge with Master Data to convert quantities in weight
3. Distances Collections
Collect the distances by transportation mode
III. CO2 Calculation
1. Final Calculation of the CO2 Emissions
2. What can you do if you have missing data?
VI. Example of visualization using PowerBI

In [41]:
import pandas as pd
import warnings

In [42]:
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_columns', None)
pd.options.display.max_seq_items = 2000
warnings.filterwarnings('ignore')

In [43]:
%%html
<style>
.dataframe td {
  white-space: nowrap;
}
</style>

# Initial Datasets Import

## Import Shipped Order Lines

In [44]:
url_lines ='https://raw.githubusercontent.com/samirsaci/supply-chain-sustainability/main/Data/order_lines.csv'
df_lines = pd.read_csv(url_lines, index_col=0)
df_lines.head()

,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros
112,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47
14485,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03
14486,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45
14487,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45
14488,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16


## Import Master Data: Unit of Measure Conversions to (kg)

In [45]:
url_uom = 'https://raw.githubusercontent.com/samirsaci/supply-chain-sustainability/main/Data/uom_conversions.csv'
df_uom = pd.read_csv(url_uom, index_col=0)
df_uom.head()

# Join
df_join = df_lines.copy()
COLS_JOIN = ['Item Code']
df_join = pd.merge(df_join, df_uom, on=COLS_JOIN, how='left', suffixes=('', '_y'))
df_join.drop(df_join.filter(regex='_y$').columns.tolist(), axis=1, inplace=True)
print("{:,} records".format(len(df_join)))
df_join.head()

5,208 records


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200


## Import Distances

In [46]:
url_dist = 'https://raw.githubusercontent.com/samirsaci/supply-chain-sustainability/main/Data/distances.csv'
df_dist = pd.read_csv(url_dist, index_col=0)

# Location
df_dist['Location'] = df_dist['Customer Country'].astype(str) + ', ' + df_dist['Customer City'].astype(str)

df_dist.head()

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location
19,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES"
610,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE"
676,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF"
682,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.694,0.0,0.0,0.0,"FRANCE, BELLEVILLE"
1375,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.437,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL"


## Import Cities GPS Locations

In [47]:
url_gps = 'https://raw.githubusercontent.com/samirsaci/supply-chain-sustainability/main/Data/gps_locations.csv'
df_gps = pd.read_csv(url_gps, index_col=0)
print('{:,} Locations'.format(len(df_gps)))
df_gps.head()


19 Locations


,Location,GPS 1,GPS 2
867,"BULGARIA, DOLNI BOGROV",42.701462,23.490811
1087,"FRANCE, AUBROMETZ",50.303749,2.176058
1153,"FRANCE, BELLEVILLE",48.871184,2.386682
1371,"FRANCE, CLICHY CEDEX",48.904099,2.304123
1534,"FRANCE, GAEL",48.116682,-2.234640


# Data Processing

## Merge Distance with GPS Locations

In [48]:
df_dist = pd.merge(df_dist, df_gps, on='Location', how='left', suffixes=('', '_y'))
df_dist.drop(df_dist.filter(regex='_y$').columns.tolist(), axis=1, inplace=True)
df_dist.head()

,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950
1,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
2,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,856.000,0.0,0.0,0.0,"GERMANY, PEINE - WOLTORF",52.304594,10.312521
3,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,52.694,0.0,0.0,0.0,"FRANCE, BELLEVILLE",48.871184,2.386682
4,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5029913-13,FRANCE,MOISSY-CRAMAYEL,288.437,0.0,0.0,0.0,"FRANCE, MOISSY-CRAMAYEL",48.624666,2.595756


## Final Join to Build Records

In [49]:
COLS_JOIN = ['Warehouse Code', 'Customer Code']
df_join = pd.merge(df_join, df_dist, on = COLS_JOIN, how='left', suffixes=('', '_y'))
df_join.drop(df_join.filter(regex='_y$').columns.tolist(), axis=1, inplace=True)
print('{:,} records'.format(len(df_join)))
df_join.head()

5,208 records


,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,Warehouse Name,Warehouse Country,Warehouse City,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369


# Calculation at order line level

In [50]:
# CAlculate Weight (KG)
df_join['KG'] = df_join['Units'] * df_join['Conversion Ratio']

# Calculation at Line Level
df_line = df_join.copy()

# CO2 Emissions CAlculation
dict_co2e = dict(zip(['Air', 'Sea', 'Road', 'Rail'], [2.1, 0.01, 0.096, 0.028]))
MODES = ['Road', 'Rail', 'Sea', 'Air']
for mode in MODES:
  df_line['CO2 ' + mode] = df_line['KG'].astype(float)/1000 * df_line[mode].astype(float) * dict_co2e[mode]
df_line['CO2 Total'] = df_line[['CO2 ' + mode for mode in MODES]].sum(axis = 1)
df_line.to_csv('detailed_report.csv')
df_line.head()

,Date,Month-Year,Warehouse Code,Customer Code,Order Number,Order Line Number,Item Code,Units,Euros,Conversion Ratio,Warehouse Name,Warehouse Country,Warehouse City,Customer Country,Customer City,Road,Rail,Sea,Air,Location,GPS 1,GPS 2,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 Total
0,2021-01-04 00:00:00.000,1-2021,3403434,5002915-15,20247100,1,963543-43,2.0,381.47,56.58400,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,LES ANGLES,765.728,0.0,0.0,0.0,"FRANCE, LES ANGLES",42.579678,2.048950,113.168,8.318967,0.0,0.0,0.0,8.318967
1,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,1,8902753-53,250.0,187.03,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369,8.000,0.216183,0.0,0.0,0.0,0.216183
2,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,2,8308591-91,500.0,452.45,0.02219,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369,11.095,0.299818,0.0,0.0,0.0,0.299818
3,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,3,8308621-21,500.0,452.45,0.02219,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369,11.095,0.299818,0.0,0.0,0.0,0.299818
4,2021-01-07 00:00:00.000,1-2021,3403434,5397843-43,20258239,4,8022921-21,400.0,567.16,0.03200,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,FRANCE,SUCY EN BRIE,281.488,0.0,0.0,0.0,"FRANCE, SUCY EN BRIE",48.768810,2.537369,12.800,0.345892,0.0,0.0,0.0,0.345892


# Calculation at order level

In [51]:
# Agg by order
GPBY_ORDER = ['Date', 'Month-Year',
              'Warehouse Code', 'Warehouse Name', 'Warehouse Country', 'Warehouse City', 
              'Customer Code', 'Customer Country', 'Customer City', 'Location', 'GPS 1', 'GPS 2', 
              'Road', 'Rail', 'Sea', 'Air', 
              'Order Number']

df_agg = pd.DataFrame(df_join.groupby(GPBY_ORDER)[['Units', 'KG']].sum())
df_agg .reset_index(inplace = True)
df_agg.head() 

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,KG
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,2.0,113.16800
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,20203388,153.0,2530.32471
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.871184,2.386682,52.694,0.0,0.0,0.0,20203383,128.0,2116.87296
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,2771.0,71.45232
4,2021-01-08 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,"FRANCE, LEVALLOIS PERRET",48.894640,2.287419,295.165,0.0,0.0,0.0,21000061,2125.0,111.95200


# Calculate CO2 = f(KG, Ratios)

In [55]:
# CO2 Emissions
dict_co2e = dict(zip(['Air', 'Sea', 'Road', 'Rail'], [2.1, 0.01, 0.096, 0.028]))
MODEs = ['Road', 'Rail', 'Sea', 'Air']
for mode in MODES:
  df_agg['CO2 ' + mode] = df_agg['KG'].astype(float)/1000 * df_agg[mode].astype(float) * dict_co2e[mode]
df_agg['CO2 Total' + mode] = df_agg[['CO2 ' + mode for mode in MODES]].sum(axis = 1)
df_agg.head()  

,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 TotalAir
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,2.0,113.16800,8.318967,0.0,0.0,0.0,8.318967
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,20203388,153.0,2530.32471,207.931963,0.0,0.0,0.0,207.931963
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.871184,2.386682,52.694,0.0,0.0,0.0,20203383,128.0,2116.87296,10.708464,0.0,0.0,0.0,10.708464
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,2771.0,71.45232,1.930845,0.0,0.0,0.0,1.930845
4,2021-01-08 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,"FRANCE, LEVALLOIS PERRET",48.894640,2.287419,295.165,0.0,0.0,0.0,21000061,2125.0,111.95200,3.172254,0.0,0.0,0.0,3.172254


# Final mapping for visualization

In [60]:
# Mapping the delivery Mode
df_agg['Delivery Mode'] = df_agg[MODEs].astype(float).apply(
    lambda t: [mode if t[mode]>0 else '-' for mode in MODES], axis = 1)
dict_map = dict(zip(df_agg['Delivery Mode'].astype(str).unique(), 
    [i.replace(", '-'",'').replace("'-'",'').replace("'",'') for i in df_agg['Delivery Mode'].astype(str).unique()]))
df_agg['Delivery Mode'] = df_agg['Delivery Mode'].astype(str).map(dict_map)
df_agg.head()


,Date,Month-Year,Warehouse Code,Warehouse Name,Warehouse Country,Warehouse City,Customer Code,Customer Country,Customer City,Location,GPS 1,GPS 2,Road,Rail,Sea,Air,Order Number,Units,KG,CO2 Road,CO2 Rail,CO2 Sea,CO2 Air,CO2 TotalAir,Delivery Mode
0,2021-01-04 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5002915-15,FRANCE,LES ANGLES,"FRANCE, LES ANGLES",42.579678,2.048950,765.728,0.0,0.0,0.0,20247100,2.0,113.16800,8.318967,0.0,0.0,0.0,8.318967,[Road]
1,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340669-69,GERMANY,PEINE - WOLTORF,"GERMANY, PEINE - WOLTORF",52.304594,10.312521,856.000,0.0,0.0,0.0,20203388,153.0,2530.32471,207.931963,0.0,0.0,0.0,207.931963,[Road]
2,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5340672-72,FRANCE,BELLEVILLE,"FRANCE, BELLEVILLE",48.871184,2.386682,52.694,0.0,0.0,0.0,20203383,128.0,2116.87296,10.708464,0.0,0.0,0.0,10.708464,[Road]
3,2021-01-07 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5397843-43,FRANCE,SUCY EN BRIE,"FRANCE, SUCY EN BRIE",48.768810,2.537369,281.488,0.0,0.0,0.0,20258239,2771.0,71.45232,1.930845,0.0,0.0,0.0,1.930845,[Road]
4,2021-01-08 00:00:00.000,1-2021,3403434,WAREHOUSE PARIS AREA 1,FRANCE,CHALONS-EN-CHAMPAGNE,5003451-51,FRANCE,LEVALLOIS PERRET,"FRANCE, LEVALLOIS PERRET",48.894640,2.287419,295.165,0.0,0.0,0.0,21000061,2125.0,111.95200,3.172254,0.0,0.0,0.0,3.172254,[Road]


In [61]:
# Save Final Report
df_agg.to_csv('final_report.csv')